# Import Libraries

In [2]:
from langchain.llms.openai import OpenAIChat
from langchain.chains import ConversationChain
from langchain.globals import set_debug
from dotenv import load_dotenv
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationTokenBufferMemory,
    ConversationSummaryBufferMemory,
)

set_debug(True)  # Enable debug mode
load_dotenv()  # Load the variables from .env file

True

# Memory

LLMs are stateless, so they don't have any knowledge of the past prompts and responses. So LangChain provides a memory mechanism to store the past prompts and use them in the current prompt.

By doing this, we can make the LLM conversational.

In [3]:
llm = OpenAIChat(temperature=0.0)
memory = ConversationBufferMemory()
conversation_chain = ConversationChain(llm=llm, memory=memory)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
conversation_chain("Hello, My name is John.")

[chain/start] [1:chain:ConversationChain] Entering Chain run with input:
{
  "input": "Hello, My name is John.",
  "history": ""
}
[llm/start] [1:chain:ConversationChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: Hello, My name is John.\nAI:"
  ]
}


[llm/end] [1:chain:ConversationChain > 2:llm:OpenAIChat] [3.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello John! It's nice to meet you. How can I assist you today?",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 70,
      "completion_tokens": 17,
      "total_tokens": 87
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:ConversationChain] [3.00s] Exiting Chain run with output:
{
  "response": "Hello John! It's nice to meet you. How can I assist you today?"
}


{'input': 'Hello, My name is John.',
 'history': '',
 'response': "Hello John! It's nice to meet you. How can I assist you today?"}

In [5]:
conversation_chain("What is my name?")

[chain/start] [1:chain:ConversationChain] Entering Chain run with input:
{
  "input": "What is my name?",
  "history": "Human: Hello, My name is John.\nAI: Hello John! It's nice to meet you. How can I assist you today?"
}
[llm/start] [1:chain:ConversationChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hello, My name is John.\nAI: Hello John! It's nice to meet you. How can I assist you today?\nHuman: What is my name?\nAI:"
  ]
}
[llm/end] [1:chain:ConversationChain > 2:llm:OpenAIChat] [1.11s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Your name is John.",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_toke

{'input': 'What is my name?',
 'history': "Human: Hello, My name is John.\nAI: Hello John! It's nice to meet you. How can I assist you today?",
 'response': 'Your name is John.'}

As you can see the chain returns a dictionary with a `history` key. This key contains a list of the past prompts and responses.


## ConversationBufferWindowMemory

All the models have a `Context Window` with a fixed size of Tokens. The context window is the maximum number of tokens that the model can use in the input prompt.

The `ConversationBufferWindowMemory` is a memory that stores the past prompts and responses in a fixed size window.

In [6]:
window_memory = ConversationBufferWindowMemory(
    k=1
)  # k is the number of conversation exchanges to remember

In [7]:
window_conv = ConversationChain(llm=llm, memory=window_memory)

In [8]:
window_memory.save_context({"input": "Hi"}, {"output": "What's up"})
window_memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [9]:
window_memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

As you can see if we used `k = 1` then the context will be the last prompt and response.

## ConversationTokenBufferMemory

In [10]:
memory_token = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)

In [11]:
memory_token = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory_token.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory_token.save_context(
    {"input": "Backpropagation is what?"}, {"output": "Beautiful!"}
)
memory_token.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

In [12]:
# There are multiple exchanges truncated because of the max_token_limit
memory_token.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

The `ConversationTokenBufferMemory` tokenizes the past prompts and responses, and specifies the maximum number of tokens that the model can use in the input prompt.

## ConversationSummaryMemory

It uses a Model to summarize the past prompts and responses and specifies the maximum number of tokens that the model can use in the input prompt.

In [13]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian restaurant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
summary_memory.save_context({"input": "Hello"}, {"output": "What's up"})
summary_memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
summary_memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?"
}
[llm/start] [1:chain:LLMChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew 

In [14]:
summary_memory.load_memory_variables({})

{'history': 'System: The human greets the AI and asks what is on the schedule for the day.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian restaurant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}